In [1]:
import tensorflow as tf
import numpy as np
from tensorflow.examples.tutorials.mnist import input_data

In [2]:
# 加载数据
#mnist = input_data.read_data_sets('mnist_data/', fake_data = True)
mnist = input_data.read_data_sets("mnist_data/", one_hot=True)

print("type of 'mnist is %s'" % (type(mnist)))
print("number of train data is %d" % mnist.train.num_examples)
print("number of test data is %d" % mnist.test.num_examples)
print(mnist.train.images.shape)

Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
Instructions for updating:
Please write your own downloading logic.
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-images-idx3-ubyte.gz
Instructions for updating:
Please use tf.data to implement this functionality.
Extracting mnist_data/train-labels-idx1-ubyte.gz
Instructions for updating:
Please use tf.one_hot on tensors.
Extracting mnist_data/t10k-images-idx3-ubyte.gz
Extracting mnist_data/t10k-labels-idx1-ubyte.gz
Instructions for updating:
Please use alternatives such as official/mnist/dataset.py from tensorflow/models.
type of 'mnist is <class 'tensorflow.contrib.learn.python.learn.datasets.base.Datasets'>'
number of train data is 55000
number of test data is 10000
(55000, 784)


In [3]:
# 定义输入输出
input_image = tf.placeholder(dtype=tf.float32, shape=[None, 784], name='input_image')
input_label = tf.placeholder(dtype=tf.float32, shape=[None, 10], name='input_label')


In [4]:
# 定义网络
hidden1_units = 20
hidden2_units = 35
def dnn_model(input, num_classes=10, is_training=True, dropout_keep_prob=0.5, scope='dnn_model'):
    _, dim = input.shape
    print('input shape is ', dim)
    # hidden 1
    with tf.name_scope('hidden_L1'):
        weight_L1 = tf.Variable(tf.truncated_normal([784, hidden1_units]), name='weight_L1')
        bias_L1 = tf.Variable(tf.zeros([1, hidden1_units]), name='bias_L1')
        operator1 = tf.matmul(input_image, weight_L1) + bias_L1
        ## activation function
        hidden1_output = tf.nn.relu(operator1)
    
    # hidden2
    with tf.name_scope('hidden_L2'):
        weight_L2 = tf.Variable(tf.truncated_normal([hidden1_units, hidden2_units]), name='weight_L2')
        bias_L2 = tf.Variable(tf.zeros([1, hidden2_units]), name='bias_L2')
        operator2 = tf.matmul(hidden1_output, weight_L2) + bias_L2
        ## activation function
        hidden2_output = tf.nn.relu(operator2)
            
    # fully connected , softmax
    with tf.name_scope('logits'):
        weight_L3 = tf.Variable(tf.truncated_normal([hidden2_units, num_classes]), name='weight_L3')
        bias_L3 = tf.Variable(tf.zeros([1, num_classes]), name='bias_L3')
        operator_L3 = tf.matmul(hidden2_output, weight_L3) + bias_L3
        ## dropout 
        logits = tf.nn.dropout(operator_L3, keep_prob=dropout_keep_prob)
        return logits

In [5]:
# 定义损失函数
def loss_function(logits, labels):
    predict = tf.nn.softmax(logits)
    # 交叉墒
    loss = tf.reduce_mean(-tf.reduce_sum(labels * tf.log(predict), axis=1))
    return loss
    
def loss_function2(logits, labels):
    loss = tf.nn.softmax_cross_entropy_with_logits(labels=labels, logits=logits)
    tf.summary.scalar('loss', loss)
    return tf.reduce_mean(loss)

In [6]:
# 定义train函数
def train(loss, learning_rate=0.2):
    global_step = tf.Variable(0, name='global_step', trainable=False)
#     optimize = tf.train.GradientDescentOptimizer(learning_rate)
    optimize = tf.train.AdamOptimizer(learning_rate=learning_rate)
    train_op = optimize.minimize(loss,global_step=global_step)
    
    return global_step, train_op

In [7]:
def evaluation(logits, labels):
    labels = tf.cast(labels, tf.int32)
    # For a classifier model, we can use the in_top_k Op.
    # It returns a bool tensor with shape [batch_size] that is true for
    # the examples where the label is in the top k (here k=1)
    # of all logits for that example.
    print('logits and labels shape is ', logits.shape, labels.shape)
    print('logits and labels type is ', logits.dtype, logits.dtype)
    labels = tf.argmax(labels, axis=1)
    correct = tf.nn.in_top_k(logits, labels, 1)
    print('correct shape is ', correct.shape)

    # Return the number of true entries.
    return tf.reduce_mean(tf.cast(correct, tf.float32), axis= -1)
#     return tf.cast(correct, tf.int32)

In [8]:
# 模型运行
logits = dnn_model(input_image)
loss = loss_function2(logits, input_label)
step, train_op = train(loss, 0.01)
eval_op = evaluation(logits, input_label)
init_global = tf.global_variables_initializer()
init_local = tf.local_variables_initializer()
merged_summary_op = tf.summary.merge_all()
with tf.Session() as sess:
    sess.run(init_global)
    sess.run(init_local)
    summary_writer = tf.summary.FileWriter(logdir="log/",graph=sess.graph)
    for index in range(50000):
        image_batch, label_batch  = mnist.train.next_batch(100)
        test_batch, test_label_batch = mnist.test.next_batch(1000)
        _, rstep, summary_step, rloss = sess.run([train_op, step, merged_summary_op,  loss], feed_dict={
            input_image: image_batch,
            input_label: label_batch
        })
        summary_writer.add_summary(summary_str, rstep)
        if index % 2000 == 0:
            accuracy = sess.run(eval_op, feed_dict={
                input_image: test_batch,
                input_label: test_label_batch
            })
            print('{} times : step {}: loss is {}, accuracy is {}'.format(index, rstep,  rloss, accuracy))
        
        

input shape is  784
Instructions for updating:
Colocations handled automatically by placer.
Instructions for updating:
Please use `rate` instead of `keep_prob`. Rate should be set to `rate = 1 - keep_prob`.
Instructions for updating:

Future major versions of TensorFlow will allow gradients to flow
into the labels input on backprop by default.

See `tf.nn.softmax_cross_entropy_with_logits_v2`.

logits and labels shape is  (?, 10) (?, 10)
logits and labels type is  <dtype: 'float32'> <dtype: 'float32'>
correct shape is  (?,)


InvalidArgumentError: tags and values not the same shape: [] != [100] (tag 'loss')
	 [[node loss (defined at <ipython-input-5-13f61ea16c3d>:10) ]]

Caused by op 'loss', defined at:
  File "/usr/lib/python3.7/runpy.py", line 193, in _run_module_as_main
    "__main__", mod_spec)
  File "/usr/lib/python3.7/runpy.py", line 85, in _run_code
    exec(code, run_globals)
  File "/usr/lib/python3.7/site-packages/ipykernel_launcher.py", line 16, in <module>
    app.launch_new_instance()
  File "/usr/lib/python3.7/site-packages/traitlets/config/application.py", line 658, in launch_instance
    app.start()
  File "/usr/lib/python3.7/site-packages/ipykernel/kernelapp.py", line 505, in start
    self.io_loop.start()
  File "/usr/lib/python3.7/site-packages/tornado/platform/asyncio.py", line 148, in start
    self.asyncio_loop.run_forever()
  File "/usr/lib/python3.7/asyncio/base_events.py", line 539, in run_forever
    self._run_once()
  File "/usr/lib/python3.7/asyncio/base_events.py", line 1775, i